<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-19 14:24:40
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.37 C
-------------------
Today PnL: 1.04 L (0.76%)
Current PnL: -17.60 L (-11.97%)
CY Booked + Current PnL: -9.33 L (-6.34%)
-------------------
Total profit:  3.75 L
Total loss:  -21.35 L
-------------------
Total Booked + Current PnL: 17.65 L (14.71%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.02%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.22 L (56.25%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.79,113144.0,8832.0,7818.0,-0.98,8.47,6.91,15.96,9.0,1.13,0.83,26.80,X5K,ATH,METALS
52,LALPATHLAB,2838.53,3545.00,-22.21,H-SC,11.65,232519.0,39499.0,8533.0,-0.27,20.46,3.67,24.89,137.0,4.63,1.71,48.59,X40N,NTT,HEALTHCARE
69,SBIN,760.30,863.00,-11.34,M-LC,3.76,219791.0,18312.0,8902.0,0.21,9.09,4.05,13.51,60.0,2.06,1.61,22.83,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.59,172260.0,16083.0,10008.0,0.22,10.30,5.81,16.71,22.0,1.61,1.26,29.53,X40N,BTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,11.51,H-MC,13.95,159175.0,32975.0,13196.0,2.16,26.13,8.29,36.59,108.0,2.50,1.17,41.92,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.22,H-SC,7.61,102610.0,110.0,40716.0,0.96,0.11,39.68,39.83,114.0,0.00,0.75,1.59,AR,ATH,MISC
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.53,178716.0,-1092.0,113503.0,1.17,-0.61,63.51,62.52,88.0,-0.01,1.31,33.27,XR,NTT,BANKS
59,PGHH,13388.00,18062.58,-31.21,H-MC,6.12,199140.0,-1680.0,71790.0,-0.76,-0.84,36.05,34.92,80.0,-0.02,1.46,3.80,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101262.0,758.0,29882.0,1.45,0.75,29.51,30.49,79.0,0.03,0.74,33.40,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-37.92,H-LC,2.95,148785.0,-1147.0,15741.0,0.60,-0.76,10.58,9.74,4.0,-0.07,1.09,5.43,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-20.12,H-LC,3.08,212830.0,8330.0,33095.0,-0.75,4.07,15.55,20.25,10.0,0.25,1.56,13.16,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-12.30,H-LC,3.46,269896.0,4470.0,52333.0,0.81,1.68,19.39,21.40,11.0,0.09,1.98,8.90,XY25,NTT,FMCG
54,LTIM,5564.16,7230.2,-7.51,H-LC,3.72,183744.0,-16566.0,76548.0,0.28,-8.27,41.66,29.94,16.0,-0.22,1.35,26.94,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-10.18,H-LC,4.20,175270.0,-26116.0,113733.0,-0.38,-12.97,64.89,43.51,27.0,-0.23,1.29,22.01,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,LAOPALA,369.40,464.00,123.28,H-SC,2.12,70830.0,-30016.0,55842.0,6.59,-29.76,78.84,25.61,142.0,-0.54,0.52,31.32,AR,NTT,CERAMICS
26,DEN,52.79,75.00,112.44,M-SC,2.73,59584.0,-24880.0,60418.0,5.11,-29.46,101.40,42.07,237.0,-0.41,0.44,26.54,AR,NTT,ENTERTAINMENT
67,SAMMAANCAP,170.35,326.00,-164.77,M-SC,6.64,73980.0,-28230.0,121623.0,4.04,-27.62,164.40,91.37,208.0,-0.23,0.54,20.09,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-7.17,H-SC,8.12,201190.0,-47340.0,262814.0,3.76,-19.05,130.63,86.70,151.0,-0.18,1.48,34.31,XY24,NTT,BANKS
51,KPIGREEN,497.21,731.35,12.86,H-SC,11.49,134606.0,9309.0,49697.0,3.70,7.43,36.92,47.09,141.0,0.19,0.99,68.47,MH,ATH,POWER


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,SYMPHONY,1306.42,1306.00,-41.14,M-SC,13.87,125629.0,-45512.0,45453.0,-1.90,-26.59,36.18,-0.03,196.0,-1.00,0.92,1.56,OX40N,NTT,DURABLES
75,STARHEALTH,551.56,761.00,7.30,H-SC,1.59,159538.0,-40678.0,116702.0,-1.60,-20.32,73.15,37.97,144.0,-0.35,1.17,28.10,XY24,NTT,INSURANCE
19,CAMS,3643.00,5211.76,-5.61,H-SC,4.64,107425.0,5421.0,38501.0,-1.54,5.31,35.84,43.06,122.0,0.14,0.79,23.34,X40N,ATH,MISC
76,SURYODAY,189.78,240.00,40.35,H-SC,5.21,100314.0,-49612.0,89289.0,-1.51,-33.09,89.01,26.46,135.0,-0.56,0.74,28.51,XR,NTT,BANKS
18,CAMPUS,294.86,393.00,-28.70,M-SC,2.92,148424.0,-19351.0,75192.0,-1.34,-11.53,50.66,33.28,210.0,-0.26,1.09,14.91,XY24,NTT,FOOTWEAR


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,101.92,M-SC,9.85,86991.0,-13786.0,13884.0,3.11,-13.68,15.96,0.09,245.0,-0.99,0.64,14.29,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,120.42,M-SC,2.26,144547.0,-30012.0,30051.0,2.25,-17.19,20.79,0.02,221.0,-1.00,1.06,25.34,OX40N,NTT,PAINTS
85,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.87,115911.0,-26568.0,46086.0,-0.42,-18.65,39.76,13.70,163.0,-0.58,0.85,36.81,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.14,H-SC,3.02,116910.0,-26893.0,53638.0,-0.06,-18.70,45.88,18.60,149.0,-0.50,0.86,29.01,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.34,H-SC,3.14,218790.0,-50877.0,87210.0,2.63,-18.87,39.86,13.47,138.0,-0.58,1.61,10.66,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.11,234874.0,-61060.0,154970.0,-0.37,-20.63,65.98,31.73,1.0,-0.39,1.72,0.27,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,11.59,258046.0,-3555.0,149176.0,0.40,-1.36,57.81,55.67,3.0,-0.02,1.89,4.66,X40,BTT,IT
47,ITC,411.90,452.00,-37.92,H-LC,2.95,148785.0,-1147.0,15741.0,0.60,-0.76,10.58,9.74,4.0,-0.07,1.09,5.43,X40,NTT,FMCG
90,VBL,492.64,671.64,-10.10,H-LC,9.76,273143.0,8103.0,88198.0,1.58,3.06,32.29,36.33,5.0,0.09,2.00,16.76,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.81,H-LC,11.27,197574.0,-18945.0,111847.0,0.57,-8.75,56.61,42.91,7.0,-0.17,1.45,3.47,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-22.21,H-SC,11.65,232519.0,39499.0,8533.0,-0.27,20.46,3.67,24.89,137.0,4.63,1.71,48.59,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,11.51,H-MC,13.95,159175.0,32975.0,13196.0,2.16,26.13,8.29,36.59,108.0,2.50,1.17,41.92,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-9.07,H-LC,7.18,227174.0,42693.0,27670.0,1.72,23.14,12.18,38.14,38.0,1.54,1.67,36.20,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,12.86,H-SC,11.49,134606.0,9309.0,49697.0,3.70,7.43,36.92,47.09,141.0,0.19,0.99,68.47,MH,ATH,POWER
56,MEDANTA,1087.93,1486.00,11.51,H-MC,13.95,159175.0,32975.0,13196.0,2.16,26.13,8.29,36.59,108.0,2.50,1.17,41.92,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,10.86,H-MC,6.24,191420.0,26750.0,41213.0,-0.08,16.24,21.53,41.28,89.0,0.65,1.40,25.52,X40N,ATH,AC
91,VOLTAS,1278.28,1918.49,-1.92,H-MC,7.60,205710.0,13968.0,82058.0,1.65,7.28,39.89,50.08,99.0,0.17,1.51,14.58,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179655.0,28742.0,28565.0,2.99,19.05,15.90,37.97,93.0,1.01,1.32,49.17,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179655.0,28742.0,28565.0,2.99,19.05,15.90,37.97,93.0,1.01,1.32,49.17,AR,ATH,AUTO
51,KPIGREEN,497.21,731.35,12.86,H-SC,11.49,134606.0,9309.0,49697.0,3.70,7.43,36.92,47.09,141.0,0.19,0.99,68.47,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-50.72,H-SC,11.88,137604.0,14268.0,119495.0,0.91,11.57,86.84,108.45,119.0,0.12,1.01,36.51,AR,ATH,MISC
92,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,4.77,96375.0,4876.0,73871.0,-1.15,5.33,76.65,86.07,223.0,0.07,0.71,39.42,XR,NTT,DURABLES
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101262.0,758.0,29882.0,1.45,0.75,29.51,30.49,79.0,0.03,0.74,33.40,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.43,74534.0,-39015.0,79066.0,1.97,-34.36,106.08,35.27,268.0,-0.49,0.55,82.42,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,6883.33,L-SC,6.97,76630.0,-17180.0,93795.0,2.61,-18.31,122.40,81.67,269.0,-0.18,0.56,48.40,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-2.82,H-SC,15.49,90476.0,-10487.0,105015.0,0.88,-10.39,116.07,93.63,148.0,-0.10,0.66,34.76,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-17.34,H-SC,12.72,93705.0,-4275.0,26144.0,1.81,-4.36,27.90,22.32,152.0,-0.16,0.69,35.90,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-324.58,H-SC,16.02,95394.0,-37892.0,122486.0,-0.89,-28.43,128.40,63.47,139.0,-0.31,0.70,59.90,XR,NTT,CHEMICALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179655.0,28742.0,28565.0,2.99,19.05,15.90,37.97,93.0,1.01,1.32,49.17,AR,ATH,AUTO
52,LALPATHLAB,2838.53,3545.00,-22.21,H-SC,11.65,232519.0,39499.0,8533.0,-0.27,20.46,3.67,24.89,137.0,4.63,1.71,48.59,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-9.07,H-LC,7.18,227174.0,42693.0,27670.0,1.72,23.14,12.18,38.14,38.0,1.54,1.67,36.20,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-10.18,H-LC,4.20,175270.0,-26116.0,113733.0,-0.38,-12.97,64.89,43.51,27.0,-0.23,1.29,22.01,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.59,172260.0,16083.0,10008.0,0.22,10.30,5.81,16.71,22.0,1.61,1.26,29.53,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.47
1,25,40.99
2,50,70.50


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.58
LC    35.41
MC    26.00
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.18
X40N     15.30
X40      14.08
XY25     11.58
XR        8.85
AR        8.30
OX40N     7.33
X5K       2.20
MH        1.73
X200      1.35
SR        1.09
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.25
H-SC    24.02
H-MC    23.12
M-SC    13.10
M-LC     7.11
M-MC     2.55
L-SC     1.46
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.14,-0.52,31.31
IT,11.53,-18.56,77.77
FINANCE,8.01,-19.39,63.56
BANKS,7.46,-12.15,59.77
MISC,5.74,-17.58,84.97
PAINTS,5.59,-14.83,37.46
HEALTHCARE,5.16,6.67,16.89
ELECTRICAL,4.66,-10.77,58.74
AUTO,4.55,-8.99,56.41


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2797141.0,23
XR,1089113.0,13
AR,1077313.0,10
X40,818101.0,11
X40N,520807.0,12
XY25,471165.0,8
OX40N,458345.0,10
SR,194380.0,2
X5K,139038.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2745263.0,28
M-SC,1459350.0,17
H-MC,1227814.0,18
H-LC,1186259.0,19
M-LC,409095.0,5
M-MC,330067.0,2
L-SC,261917.0,3
L-MC,57694.0,1
L-LC,44071.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926091.0,6
M-SC,XY24,860356.0,7
H-SC,AR,659034.0,5
H-LC,X40,529207.0,6
H-SC,XR,516132.0,6
H-MC,XY24,478643.0,5
M-MC,XY24,330067.0,2
H-LC,AR,232693.0,2
H-MC,X40,208973.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
